### Bert-base raw model

In [15]:

import pandas as pd
import numpy as np
import torch
import transformers

from transformers import BertForMaskedLM
from transformers import BertTokenizer
from transformers import BertModel

In [16]:
# HYPERPARAMS
SEED_SPLIT = 0
SEED_TRAIN = 0

MAX_SEQ_LEN = 128
TRAIN_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5 
LR_WARMUP_STEPS = 100
WEIGHT_DECAY = 0.01

In [17]:
MODEL = 'bert'
bert_type = 'bert-base-uncased'
TokenizerClass = BertTokenizer
# ModelClass = BertForMaskedLM 
ModelClass = BertModel 



# tokenizer = TokenizerClass.from_pretrained(
#             bert_type, use_fast=True, do_lower_case=False, max_len=MAX_SEQ_LEN
#             )
# model = ModelClass.from_pretrained(bert_type)


tokenizer = TokenizerClass.from_pretrained(
            './model', use_fast=True, do_lower_case=False, max_len=MAX_SEQ_LEN
            )
model = ModelClass.from_pretrained('./model')


model.eval()
print(len(tokenizer))

# tokenizer = BertTokenizer.from_pretrained('data/cased_L-12_H-768_A-12') #改为自己存放模型的目录
# model = BertModel.from_pretrained('data/cased_L-12_H-768_A-12')

Some weights of the model checkpoint at ./model were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN t

31370


In [18]:

vocab = open('./model/vocab.txt', 'r')
vocab_wds = [line.strip('\n') for line in vocab]

def encode_phrase(phrase):
#     phrase = "Your English phrase goes here."
    tokens = tokenizer.encode_plus(phrase, add_special_tokens=True, return_tensors='pt')
#     print(tokens['input_ids'])
    ids = tokens['input_ids'].numpy().tolist()
    id_char = ""
    for i in ids[0]:
        if i>= len(vocab_wds):
            id_char += "{0}({1}), ".format(i, "@@@")
        else:
            id_char += "{0}({1}), ".format(i, vocab_wds[i])
#     print(id_char)
    with torch.no_grad():
        outputs = model(tokens['input_ids'], tokens['attention_mask'])
        phrase_vector = outputs.last_hidden_state.mean(dim=1).squeeze()
    return phrase_vector

    
# print(phrase_vector.shape)

In [19]:
vec = encode_phrase("rplidar")
# print(vec)

In [20]:
def cos_similar(sen_a_vec, sen_b_vec):
    vector_a = np.mat(sen_a_vec)
    vector_b = np.mat(sen_b_vec)
    num = float(vector_a*vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num/denom
    return cos


In [21]:
def embedding_similarity(p1,p2):
    v1 = encode_phrase(p1)
    v2 = encode_phrase(p2)
    sim = cos_similar(v1, v2)
    return sim

In [26]:
s1 = "bringup rplidar"
# s2 = "The rplidar ros package support rplidar A2/A1"
s2 = "start rplidar"
# tensor([[  101,   100,  1054, 24759,  8524(##ida),  2099, 20996,  2015,  7427,  2490,
#           1054, 24759,  8524,  2099,   100,  1013,   100,   102]])
print(embedding_similarity(s1, s2))
s2 = "rospeex"
print(embedding_similarity(s1, s2))
s1 = "turtlebot"
s2 = "turtlebot2"
s3 = "turtlebot3"
print(embedding_similarity(s1, s2))
print(embedding_similarity(s2, s1))
print(embedding_similarity(s1, s3))
print(embedding_similarity(s2, s3))

0.8037486679493062
0.8525486070707025
0.9539678352596432
0.9539678352596432
0.9361038749221334
0.9148580807619587


In [29]:
s1 = "rplidar"
# s2 = "The rplidar ros package support rplidar A2/A1"
s2 = "rplidar"
# tensor([[  101,   100,  1054, 24759,  8524(##ida),  2099, 20996,  2015,  7427,  2490,
#           1054, 24759,  8524,  2099,   100,  1013,   100,   102]])
print(embedding_similarity(s1, s2))


# s2 = "The rplidar ros package support rplidar A2/A1"
s2 = "start rplidar sensor"

print(embedding_similarity(s1, s2))



1.000000101597313
0.5860748703037677
